# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.

In [ ]:
def get_IG(v, left, right) :
    r_v, r_left, r_right = len(v), len(left), len(right)
    return entropy(v) - (r_left * entropy(left) + r_right * entropy(right)) / r_v


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

In [ ]:
class node :
    def __init__(self, data = None, target = None):
        self.data = data
        self.target = target
        self.best_split_param = None
        self.split_value = None
        self.left = None
        self.right = None
        self.item_class = None

    def indicate(self, item) :
        return item[self.best_split_param] < self.split_value

In [ ]:
from scipy.stats import mode

In [ ]:
class decision_tree :
    def __init__(self, max_depth = 5, min_leaf_size = 5, max_leaf_count = 5):
        self.root = None
        self.max_depth = max_depth
        self.min_leaf_size = min_leaf_size
        self.max_leaf_count = max_leaf_count
    
#choose the best splitting {
    def get_param_split(self, node, param) :
        grid = np.linspace(node.data[param].min(), node.data[param].max(), 10)
        best_IG = -1
        best_value = grid[0]

        for value in grid :
            cond = node.data[param] < value

            IG = get_IG(node.target, node.target[cond], node.target[~cond])

            if IG > best_IG :
                best_value, best_IG = value, IG

        return best_value, best_IG

    def get_best_split(self, node, k_params) :
        best_IG = -1
        best_value = None
        best_param = None

        if k_params is not None :
            params = np.random.choice(node.data.columns, k_params)
        else :
            params = node.data.columns

        for param in params :
            value, IG = self.get_param_split(node, param)

            if IG > best_IG :
                best_value, best_IG, best_param = value, IG, param

        return best_param, best_value
#}

#check stopping criterions {
    def get_depth(self, node) :
        if node is None :
            return 0
        return 1 + max(self.get_depth(node.left), self.get_depth(node.right))

    def get_node_size(self, node) :
        if node is None :
            return 0
        return len(node.target)

    def get_leaf_count(self, node) :
        if node is None :
            return 0
        if node.left is None and node.right is None :
            return 1
        return self.get_leaf_count(node.left) + self.get_leaf_count(node.right)

    def is_stop_criterion_violated(self, criterion, node) :
        is_stop = (self.get_node_size(node.left) == 0 or 
                   self.get_node_size(node.right) == 0)
        if not is_stop :
            if criterion == 'max_depth':
                is_stop = self.get_depth(self.root) > self.max_depth
            elif criterion == 'min_leaf_size':
                is_stop = (self.get_node_size(node.left) < self.min_leaf_size or 
                          self.get_node_size(node.right) < self.min_leaf_size)
            elif criterion == 'max_leaf_count':
                is_stop = self.get_leaf_count(self.root) > self.max_leaf_count
        return is_stop

    def is_purity(self, criterion, node) :
        if criterion == 'purity' :
            return len(set(node.target)) == 1
        return False
#}

#build decision_tree {
    def create_node(self, data, target) :
        return node(data, target)

    def make_terminal(self, node):
        if node is not None and node.target is not None :
            node.item_class = mode(node.target).mode[0]
        
    def fit(self, X_train, y_train, criterion, node = None, k_params = None) : #X_train is database
        if self.root is None : 
            self.root = self.create_node(X_train, y_train)
            node = self.root

        if not self.is_purity(criterion, node) :
            best_split_param, split_value = self.get_best_split(node, k_params)
            best_split_cond = node.data[best_split_param] < split_value

            node.left = self.create_node(node.data[best_split_cond], 
                                        node.target[best_split_cond])
            node.right = self.create_node(node.data[~best_split_cond], 
                                          node.target[~best_split_cond])
            
            if self.is_stop_criterion_violated(criterion, node) :
                node.left = node.right = None

        if node.left is None and node.right is None :
            self.make_terminal(node)
            return node

        node.best_split_param = best_split_param
        node.split_value = split_value

        self.fit(node.data[best_split_cond], node.target[best_split_cond], criterion, node.left)
        self.fit(node.data[~best_split_cond], node.target[~best_split_cond], criterion, node.right)

        return self
#}

#make prediction {
    def predict(self, X_test) : #X_test is database
        predicted_target = np.array(X_test.index)
        for index, item in X_test.iterrows():
            node = self.root
            
            while node.item_class is None :
                if node.indicate(item) :
                    node = node.left
                else:
                    node = node.right
            
            predicted_target[index] = node.item_class

        return predicted_target
#}

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

In [ ]:
X_train_df = pd.DataFrame(data = X_train, columns = iris['feature_names'])
X_test_df = pd.DataFrame(data = X_test, columns = iris['feature_names'])

In [ ]:
model_1 = decision_tree().fit(X_train_df, y_train, 'max_depth')
model_2 = decision_tree().fit(X_train_df, y_train, 'min_leaf_size')
model_3 = decision_tree().fit(X_train_df, y_train, 'max_leaf_count')
model_4 = decision_tree().fit(X_train_df, y_train, 'purity')

In [ ]:
y_predicted_1 = model_1.predict(X_test_df)
y_predicted_2 = model_2.predict(X_test_df)
y_predicted_3 = model_3.predict(X_test_df)
y_predicted_4 = model_4.predict(X_test_df)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
def accuracy_score(y_test, y_predicted) :
    same = sum(y_test[i] == y_predicted[i] for i in range(len(y_test)))
    return same / len(y_test)

In [ ]:
print('--- accuracy scores ---', 
      '\nmax_depth      :  ', accuracy_score(y_test, y_predicted_1), 
      '\nmin_leaf_size  :  ', accuracy_score(y_test, y_predicted_2), 
      '\nmax_leaf_count :  ', accuracy_score(y_test, y_predicted_3), 
      '\npurity         :  ', accuracy_score(y_test, y_predicted_4))

--- accuracy scores --- 
max_depth      :   1.0 
min_leaf_size  :   0.98 
max_leaf_count :   1.0 
purity         :   0.98


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Content
- RowNumber — corresponds to the record (row) number and has no effect on the output. (DROP)
- CustomerId — contains random values and has no effect on customer leaving the bank. (DROP)
- Surname — the surname of a customer has no impact on their decision to leave the bank. (DROP)
- CreditScore — can have an effect on customer churn, since a customer with a higher credit score is less likely to leave the bank.
- Geography — a customer’s location can affect their decision to leave the bank.
- Gender — it’s interesting to explore whether gender plays a role in a customer leaving the bank.
- Age — this is certainly relevant, since older customers are less likely to leave their bank than younger ones.
- Tenure — refers to the number of years that the customer has been a client of the bank. Normally, older clients are more loyal and less likely to leave a bank.
- Balance — also a very good indicator of customer churn, as people with a higher balance in their accounts are less likely to leave the bank compared to those with lower balances.
- NumOfProducts — refers to the number of products that a customer has purchased through the bank.
- HasCrCard — denotes whether or not a customer has a credit card. This column is also relevant, since people with a credit card are less likely to leave the bank.
- IsActiveMember — active customers are less likely to leave the bank.
- EstimatedSalary — as with balance, people with lower salaries are more likely to leave the bank compared to those with higher salaries.
- Exited — whether or not the customer left the bank.

In [ ]:
churn = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/churn.csv')
churn = churn.drop(['RowNumber','CustomerId','Surname'], axis = 1)
churn.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5193,562,Spain,Female,29,9,120307.58,1,1,1,6795.61,0
8298,644,Germany,Female,51,4,95560.04,1,0,0,72628.84,1
6793,623,Germany,Male,50,7,126608.37,1,0,1,645.61,1
7934,629,Spain,Male,31,6,132876.55,1,1,1,130862.11,0
5452,558,France,Female,45,1,153697.53,2,0,0,89891.40,1


Предобработка категориальных признаков
- One-hot кодирование - не самая хорошая идея для деревьев. Мы порождаем много новых признаков, вследсвие чего сложнее становится выбор предиката на каждом шаге.

- Label encoding - совсем странная штука, так как в ней просто текстовые значения заменяются числовыми.

- Нужно использовать разновидность target encoding

In [ ]:
import category_encoders
from category_encoders import TargetEncoder

encoder = TargetEncoder()
param_to_encode = ['Geography', 'Gender']

churn[param_to_encode] = encoder.fit_transform(churn[param_to_encode], churn['Exited'])
churn.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3465,692,0.324432,0.250715,43,2,69014.49,2,0,0,164621.43,0
6228,663,0.324432,0.164559,42,5,90248.79,1,1,1,79169.73,0
2091,661,0.161548,0.250715,37,5,136425.18,1,1,0,81102.81,0
9007,726,0.161548,0.164559,31,9,0.00,2,1,1,106117.30,0
2613,630,0.161548,0.164559,26,7,129837.72,2,0,1,197001.15,0


Проверим данные на пропуски значений


In [ ]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  float64
 2   Gender           10000 non-null  float64
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 859.5 KB


Анализируем результата выполнения команды:

- 10000 строк (entries)
- 11 столбцов (Data columns)

Теперь в данных присутствует всего два типа dtypes:
- int64 - целое число (7 столбцов)
- float64 - дробное число (4 столбца)

Цифры в каждой строчке обозначают количество заполненных (non-null) значений. Так как эти цифры в каждой строчке совпадают с числом строк (10000), то в данных нет пропусков и можно двигаться дальше.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator

In [ ]:
class random_forest(BaseEstimator) :
    def __init__(self, n_trees = 5, criterion = 'entropy', 
                 max_depth = 20, min_leaf_size = 2, max_leaf_count = 10):
        self.n_trees = n_trees
        self.trees = []
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_leaf_size = min_leaf_size
        self.max_leaf_count = max_leaf_count

    def bootstrap(self, X, y) :
        m = len(X)
        indexes = np.random.choice(m, m, replace = True)
        return X.iloc[indexes, :], y.iloc[indexes]

    def fit(self, X, y) :
        for i in range(self.n_trees) :
            X_with_repl, y_with_repl = self.bootstrap(X, y)
            tree = DecisionTreeClassifier(max_depth = self.max_depth,
                                          min_samples_leaf=self.min_leaf_size,
                                          criterion = self.criterion,
                                          max_leaf_nodes = self.max_leaf_count,
                                          max_features = 'sqrt')
            self.trees.append(tree.fit(X_with_repl, y_with_repl))
        return self

    def predict(self, X) :
        predictions = np.array([tree.predict(X) for tree in self.trees])
        y_predicted = [mode(predict).mode[0] for predict in predictions.T]
        return y_predicted

In [ ]:
X = churn.drop('Exited', axis = 1)
y = churn['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = random_forest().fit(X_train, y_train)
y_predicted = model.predict(X_test)

print('-- accuracy score --\n', accuracy_score(y_test, y_predicted))

-- accuracy score --
 0.8606060606060606


Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
forest = random_forest()

forest_params = {
    'n_trees': np.arange(5, 101, 5),
    'max_depth': np.arange(2, 21, 2), 
    #'min_leaf_size': np.arange(1, 10, 2), 
    #'criterion': ['entropy', 'gini'],
    #'max_leaf_count': np.arange(2, 20, 2)
}

forest_grid = GridSearchCV(forest, forest_params, scoring = 'accuracy')

Из-за долгой работы сократила число варьируемых параметров тут

In [ ]:
%time
forest_grid.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


GridSearchCV(cv=None, error_score=nan,
             estimator=random_forest(criterion='entropy', max_depth=20,
                                     max_leaf_count=10, min_leaf_size=2,
                                     n_trees=5),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20]),
                         'n_trees': array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
forest_grid.score(X_test, y_test)

0.8463636363636363

In [ ]:
forest_grid.best_estimator_

random_forest(criterion='entropy', max_depth=16, max_leaf_count=10,
              min_leaf_size=2, n_trees=5)

In [ ]:
forest_grid.best_params_

{'max_depth': 16, 'n_trees': 5}

Когда все отработало, начинаем увеличивать параметры для подбора (большее количество требует более часа)

In [ ]:
forest1 = random_forest()

forest_params1 = {
    'n_trees': np.arange(2, 50, 5),
    'max_depth': np.arange(2, 20, 4), 
    'min_leaf_size': np.arange(1, 20, 5), 
    'criterion': ['entropy', 'gini'],
    'max_leaf_count': np.arange(2, 20, 4)
}

forest_grid1 = GridSearchCV(forest1, forest_params1, scoring = 'accuracy')

In [ ]:
%time
forest_grid1.fit(X_train, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs


GridSearchCV(cv=None, error_score=nan,
             estimator=random_forest(criterion='entropy', max_depth=20,
                                     max_leaf_count=10, min_leaf_size=2,
                                     n_trees=5),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': array([ 2,  6, 10, 14, 18]),
                         'max_leaf_count': array([ 2,  6, 10, 14, 18]),
                         'min_leaf_size': array([ 1,  6, 11, 16]),
                         'n_trees': array([ 2,  7, 12, 17, 22, 27, 32, 37, 42, 47])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
forest_grid1.score(X_test, y_test)

0.860909090909091

In [ ]:
forest_grid1.best_estimator_

random_forest(criterion='gini', max_depth=18, max_leaf_count=18,
              min_leaf_size=1, n_trees=27)

In [ ]:
forest_grid1.best_params_

{'criterion': 'gini',
 'max_depth': 18,
 'max_leaf_count': 18,
 'min_leaf_size': 1,
 'n_trees': 27}

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
forest_grid1.best_estimator_.trees[0].feature_importances_

array([0.01070917, 0.10206112, 0.        , 0.2472615 , 0.0098217 ,
       0.13312706, 0.44177772, 0.        , 0.0459421 , 0.00929962])

In [ ]:
best_model_trees = forest_grid1.best_estimator_.trees
n_trees = len(best_model_trees)

importances_array = sum(best_model_trees[i].feature_importances_ for i in range(n_trees)) / n_trees

In [ ]:
feature_importances = pd.DataFrame(index = X.columns, data = {'Importance' : importances_array})

feature_importances = feature_importances.sort_values(by = 'Importance', ascending = False)
feature_importances

,Importance
Age,0.377556
NumOfProducts,0.318074
IsActiveMember,0.106027
Geography,0.082424
Balance,0.075020
CreditScore,0.013487
EstimatedSalary,0.012089
Gender,0.009937
Tenure,0.005140
HasCrCard,0.000247


In [ ]:
importances_array.mean()

0.09999999999999999

На практике задают количество оставляемых параметров, потому нельзя сказать определенно, какие параметры нужно убрать. а какие оставить

Будем считать наиболее значимыми те параметры, для которых рассчитанное значение не меньше медианного значения более чем на треть.

In [ ]:
feature_importances[feature_importances['Importance'] >= 2 / 3 * importances_array.mean()]

,Importance
Age,0.377556
NumOfProducts,0.318074
IsActiveMember,0.106027
Geography,0.082424
Balance,0.075020


Наиболее значимые параметры
- Age — this is certainly relevant, since older customers are less likely to leave their bank than younger ones.
- NumOfProducts — refers to the number of products that a customer has purchased through the bank. (чем больше продуктов открыто у человека, тем меньше вероятность, что он решит их закрывать)
- IsActiveMember — active customers are less likely to leave the bank.
- Geography — a customer’s location can affect their decision to leave the bank. (переезд или просто неудобное расположение по отношению к банку могут стать причиной ухода из него)
- Balance — also a very good indicator of customer churn, as people with a higher balance in their accounts are less likely to leave the bank compared to those with lower balances.
